In [ ]:
import os
import numpy as np
import matplotlib.pylab as plt
from tqdm import tqdm

In [ ]:
import tensorflow as tf

In [ ]:
datadir = '/lcrc/project/CDIdefect/aicdi_generate_defect_data/'

sets = [51,52]

In [ ]:
resize_shape = 40
savedir = '/lcrc/project/AutoPhase/CDInodefect_resized%d_centered/' %resize_shape

if not os.path.isdir(savedir):
    os.mkdir(savedir)

In [ ]:
def load_normalize_resize_obj_amp(filename,shape=resize_shape):
        
    data = np.load(filename)
    
    h,w,t = data.shape
    shape = shape//2
    
    maxpos=np.unravel_index(np.abs(data).argmax(),data.shape) #Gives you index of max as a tuple
#     print(maxpos)

#     maxpos = [h//2,w//2,t//2]
    
    diff_s = data[maxpos[0]-shape:maxpos[0]+shape,maxpos[1]-shape:maxpos[1]+shape,maxpos[2]-shape:maxpos[2]+shape]
    
    obj_s = np.fft.ifftshift(np.fft.ifftn(np.fft.ifftshift(diff_s)))
    
    obj = np.zeros((h,w,t),dtype='complex')
    obj[h//2-shape:h//2+shape,w//2-shape:w//2+shape,t//2-shape:t//2+shape] = obj_s
    
    diff = np.fft.fftshift(np.fft.fftn(obj))
    scale = np.max(np.abs(obj))
    diff = diff/scale

    return diff

In [ ]:
for set_index in sets:
    data_folder = os.path.join(datadir,'set%d_nodefect/' %set_index, 'pattern/')
    filelist = sorted(os.listdir(data_folder))
    print('number of files:%d' %len(filelist))
    
    for i in tqdm(range(len(filelist))):
        
        try:
            resized_diff = load_normalize_resize_obj_amp(os.path.join(data_folder,filelist[i]))
            savename = savedir+'set%d_' %set_index+filelist[i]
            np.save(savename,resized_diff)
            
        except:
            continue

In [ ]:
dataname_list = os.path.join(savedir,'CDI_defectFree.txt')
print(dataname_list)

if os.path.exists(dataname_list):
    os.remove(dataname_list)
    
entries = sorted(os.listdir(savedir))
print(len(entries))

a = open(dataname_list, "a")
i = 0
for entry in entries:  
    a.write(entry + os.linesep)
a.close()


# augumentation

In [ ]:
import numpy as np
import matplotlib.pylab as plt
from scipy.ndimage.measurements import center_of_mass as com
from scipy.ndimage.interpolation import shift
from scipy.ndimage.interpolation import rotate as R
from tqdm import tqdm
from joblib import Parallel, delayed
import shutil, tempfile
import os
import random

import multiprocessing as ms
import concurrent.futures

import func_oversample as func

In [ ]:
datadir = '/lcrc/project/CDIdefect/aicdi_generate_defect_data/'

setNo = [51,52]

In [ ]:
resize_shape = 32
save_folder = '/lcrc/project/AutoPhase/CDInodefect_resized%d_aug/' %resize_shape

if not os.path.isdir(save_folder):
    os.mkdir(save_folder)

In [ ]:
def crop_pad(IN,dim):
    [h,w,t] = IN.shape
    output = np.zeros(dim,dtype=IN.dtype)
    
    if h>=dim[0] and w>=dim[1] and t>=dim[2]:
        output = IN[h//2-dim[0]//2:h//2+dim[0]//2,w//2-dim[1]//2:w//2+dim[1]//2,t//2-dim[2]//2:t//2+dim[2]//2]
    else:
        ps = np.max([[h,w,t],list(dim)],axis=0)

        lb = ps//2-(np.floor(np.array([h,w,t])/2)).astype('int32')
        ub = ps//2+(np.ceil(np.array([h,w,t])/2)).astype('int32')

        IN_pad = np.zeros(ps,dtype=IN.dtype)
        IN_pad[lb[0]:ub[0],lb[1]:ub[1],lb[2]:ub[2]]=IN

        lb = ps//2-(np.floor(np.array(dim)/2)).astype('int32')
        ub = ps//2+(np.ceil(np.array(dim)/2)).astype('int32')

        output = IN_pad[lb[0]:ub[0],lb[1]:ub[1],lb[2]:ub[2]]
    return output

def Resize(IN,dim):
    # from skimage.transform import resize
    # output = resize(IN, dim, preserve_range=True, anti_aliasing=True)
    ft = np.fft.fftshift(np.fft.fftn(IN))/np.prod(IN.shape)
    ft_resize = crop_pad(ft,dim)
    output = np.fft.ifftn(np.fft.ifftshift(ft_resize))*np.prod(dim)

    return output

In [ ]:
M=64
def create_obj_os(file,set_index=0,shape=resize_shape,save_folder=save_folder):

    filename = os.path.join(datadir,'set%d_nodefect/' %set_index, 'pattern/',file)
    #over sampling list
    os_list = [[1,1,1], [2,1,1],[1,2,1],[1,1,2],[1,2,2],[2,1,2],[2,2,1]]
    name = str(filename).split('.')[0]
    shape = shape//2
    
    for jj in range(len(os_list)):
        
        data = np.load(os.path.join(filename))
        
        h,w,t = data.shape
        
        maxpos = [h//2, w//2, t//2]
        diff = data[maxpos[0]-shape:maxpos[0]+shape,maxpos[1]-shape:maxpos[1]+shape,maxpos[2]-shape:maxpos[2]+shape]
        
        obj = np.fft.ifftshift(np.fft.ifftn(np.fft.ifftshift(diff)))
        s = obj.shape[0]
        dim = [s//os for os in os_list[jj]]
        obj_resize = Resize(obj,dim)
        
        #Zero everything outside object
        obj_resize=np.where(np.abs(obj_resize)<0.1, 0, obj_resize)

        realspace = np.zeros((M,M,M),dtype = np.complex128)
        realspace[M//2-dim[0]//2:M//2+dim[0]//2,M//2-dim[1]//2:M//2+dim[1]//2,M//2-dim[2]//2:M//2+dim[2]//2] = obj_resize
        
        diff = np.fft.fftshift(np.fft.fftn(realspace))
        diff = np.complex64(diff)
        
        scale = np.max(np.abs(realspace))
        diff = diff/scale
        
        savename = save_folder+'set%d_' %set_index+file+'_%d'%(jj)
        np.save(savename,diff)

In [ ]:
def data_generator(setIndex,datadir,savedir):
    
    if isinstance(setIndex, list):
        for index_each in setIndex:
            
            data_folder = os.path.join(datadir,'set%d_nodefect/' %index_each, 'pattern/')
            filelist = sorted(os.listdir(data_folder))

            for file in tqdm(filelist):
                   
                create_obj_os(file, index_each,resize_shape,save_folder)                  
            
    else:
        data_folder = os.path.join(datadir,'set%d_nodefect/' %setIndex, 'pattern/')
        filelist = sorted(os.listdir(data_folder))

        for file in tqdm(filelist):
            
            create_obj_os(file, setIndex,resize_shape,save_folder)  

In [ ]:
data_generator(setNo,datadir,save_folder)

In [ ]:
dataname_list = os.path.join(save_folder,'CDI_defectFree.txt')
print(dataname_list)

if os.path.exists(dataname_list):
    os.remove(dataname_list)
    
entries = sorted(os.listdir(save_folder))
print(len(entries))

a = open(dataname_list, "a")

for entry in entries:  
    a.write(entry + os.linesep)
a.close()